# Thu thập dữ liệu từ nhiều nguồn (Done)

In [ ]:
# Import các thư viện cần thiết
import pandas as pd

In [1]:
# Đọc dữ liệu từ file CSV
file_path = "D:/Download/2019-Oct.csv" 
data = pd.read_csv(file_path)

# Hiển thị 5 dòng đầu tiên của dữ liệu
print(data.head())

                event_time event_type  product_id          category_id  \
0  2019-10-01 00:00:00 UTC       view    44600062  2103807459595387724   
1  2019-10-01 00:00:00 UTC       view     3900821  2053013552326770905   
2  2019-10-01 00:00:01 UTC       view    17200506  2053013559792632471   
3  2019-10-01 00:00:01 UTC       view     1307067  2053013558920217191   
4  2019-10-01 00:00:04 UTC       view     1004237  2053013555631882655   

                         category_code     brand    price    user_id  \
0                                  NaN  shiseido    35.79  541312140   
1  appliances.environment.water_heater      aqua    33.20  554748717   
2           furniture.living_room.sofa       NaN   543.10  519107250   
3                   computers.notebook    lenovo   251.74  550050854   
4               electronics.smartphone     apple  1081.98  535871217   

                           user_session  
0  72d76fde-8bb3-4e00-8c23-a032dfed738c  
1  9333dfbd-b87a-4708-9857-6336556b0fc

# Gán nhãn dữ liệu cho bộ dữ liệu

In [ ]:
import pandas as pd

# Đọc dữ liệu từ tập tin (giả sử dữ liệu đã được tải vào DataFrame df)
df = pd.read_csv('path_to_your_file.csv')

# Tạo cột 'interaction' dựa trên 'event_type'
event_type_to_interaction = {
    'view': 1,
    'cart': 2,
    'purchase': 3
}

df['interaction'] = df['event_type'].map(event_type_to_interaction)

# Lọc bỏ các cột không cần thiết
df = df[['user_id', 'product_id', 'interaction']]

# Hiển thị dữ liệu đã chuẩn bị
print(df.head())
